In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "6"

In [2]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [3]:
import deep_learning

2024-06-12 12:05:14.121286: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 12:05:15.030035: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 12:05:16.374374: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
if True:
    with open("splitted_rws.pic", "rb") as r:
        X_training, X_val, X_extraction = pic.load(r)
        X_training, X_val, X_extraction = X_training[:, 200:5075], X_val[:, 200:5075], X_extraction[:, 200:5075]
    with open("splitted_labels_1000000.pic", "rb") as r:
        y_training, y_val, y_extraction = pic.load(r)
else:
    with open("traces_rws_only.pic", "rb") as r:
        traces_rws_only = pic.load(r)
    with open("labels_1000000.pic", "rb") as r:
        rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels = pic.load(r)

    X_total, y_total = deep_learning.prepare_data_dl(traces_rws_only, round_perms_labels, copy_perms_labels, round_masks_labels, rws_perms_labels, rws_masks_labels)

    profile, test = train_test_split(np.arange(X_total.shape[0]), train_size=750_000, random_state=0)

    X_profiling, X_extraction = X_total[profile], X_total[test]
    train, val = train_test_split(np.arange(X_profiling.shape[0]), test_size=0.1, random_state=0)
    X_training, X_val = X_profiling[train], X_profiling[val]

    with open("splitted_rws.pic", "wb") as w:
        pic.dump((X_training, X_val, X_extraction), w)

    if False:
        y_profiling = {}
        y_training = {}
        y_val = {}
        y_extraction = {}
        for label in y_total.keys():
            print(label, end="\r")
            y_profiling[label], y_extraction[label] = y_total[label][profile], y_total[label][test]
            y_training[label], y_val[label] = y_profiling[label][train], y_profiling[label][val]

        with open("splitted_labels_1000000.pic", "wb") as w:
            pic.dump((y_training, y_val, y_extraction), w)
    else:
        with open("splitted_labels_1000000.pic", "rb") as r:
            y_training, y_val, y_extraction = pic.load(r)


In [5]:
resnet = deep_learning.ResNetSCA(network="orig_rws_0", epochs=1000, dataset_size=X_training.shape[0])
try:
    deep_learning.check_file_exists("./resnet_models/resnet_750000_orig_rws_0.keras")
    from tensorflow.keras.models import load_model
    resnet.model = load_model("./resnet_models/resnet_750000_orig_rws_0.keras")
except ValueError:
    pass

2024-06-12 12:05:59.899728: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:da:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 12:06:00.279696: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:da:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 12:06:00.279761: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:da:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 12:06:00.293770: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:da:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 12:06:00.293854: I external/local_xla/xla/stream_executor

In [6]:
train_gen = deep_learning.DataGenerator(X_training, y_training)
val_gen = deep_learning.DataGenerator(X_val, y_val)

In [7]:
history = resnet.train_model_generator(train_gen, val_gen, "./resnet_models/resnet_750000_orig_rws_0.keras", patience=10)

Epoch 1/1000


2024-06-12 12:08:49.218535: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
/root/Pierugo/protected_side_channel_attack/.venv_linux/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1718186929.806649    4243 service.cc:145] XLA service 0x7fd1ec11f100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718186929.806715    4243 service.cc:153]   StreamExecutor device (0): NVIDIA RTX 4500 Ada Generation, Compute Capability 8.9
2024-06-12 12:08:51.880593: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc

 7559/10547 ━━━━━━━━━━━━━━━━━━━━ 5:32 111ms/step - loss: 130.0679 - rws_mask_0_0_output_accuracy: 0.1110 - rws_mask_0_1_output_accuracy: 0.0945 - rws_mask_10_0_output_accuracy: 0.1120 - rws_mask_10_1_output_accuracy: 0.0950 - rws_mask_11_0_output_accuracy: 0.1108 - rws_mask_11_1_output_accuracy: 0.0953 - rws_mask_12_0_output_accuracy: 0.1113 - rws_mask_12_1_output_accuracy: 0.0941 - rws_mask_13_0_output_accuracy: 0.1108 - rws_mask_13_1_output_accuracy: 0.0946 - rws_mask_14_0_output_accuracy: 0.1094 - rws_mask_14_1_output_accuracy: 0.0958 - rws_mask_15_0_output_accuracy: 0.1126 - rws_mask_15_1_output_accuracy: 0.0958 - rws_mask_16_0_output_accuracy: 0.1098 - rws_mask_16_1_output_accuracy: 0.0948 - rws_mask_17_0_output_accuracy: 0.1107 - rws_mask_17_1_output_accuracy: 0.0953 - rws_mask_18_0_output_accuracy: 0.1119 - rws_mask_18_1_output_accuracy: 0.0939 - rws_mask_19_0_output_accuracy: 0.1095 - rws_mask_19_1_output_accuracy: 0.0943 - rws_mask_1_0_output_accuracy: 0.1096 - rws_mask_1_1_ou

I0000 00:00:1718187878.644065    4250 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_69', 72 bytes spill stores, 72 bytes spill loads



10547/10547 ━━━━━━━━━━━━━━━━━━━━ 1389s 123ms/step - loss: 130.0746 - rws_mask_0_0_output_accuracy: 0.1111 - rws_mask_0_1_output_accuracy: 0.0944 - rws_mask_10_0_output_accuracy: 0.1120 - rws_mask_10_1_output_accuracy: 0.0950 - rws_mask_11_0_output_accuracy: 0.1107 - rws_mask_11_1_output_accuracy: 0.0952 - rws_mask_12_0_output_accuracy: 0.1112 - rws_mask_12_1_output_accuracy: 0.0942 - rws_mask_13_0_output_accuracy: 0.1109 - rws_mask_13_1_output_accuracy: 0.0946 - rws_mask_14_0_output_accuracy: 0.1092 - rws_mask_14_1_output_accuracy: 0.0955 - rws_mask_15_0_output_accuracy: 0.1123 - rws_mask_15_1_output_accuracy: 0.0956 - rws_mask_16_0_output_accuracy: 0.1098 - rws_mask_16_1_output_accuracy: 0.0949 - rws_mask_17_0_output_accuracy: 0.1107 - rws_mask_17_1_output_accuracy: 0.0952 - rws_mask_18_0_output_accuracy: 0.1119 - rws_mask_18_1_output_accuracy: 0.0939 - rws_mask_19_0_output_accuracy: 0.1094 - rws_mask_19_1_output_accuracy: 0.0943 - rws_mask_1_0_output_accuracy: 0.1095 - rws_mask_1_1_o